In [19]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import os
import torchvision.models as tmodels
from termcolor import colored
from torch.utils.tensorboard import SummaryWriter
%matplotlib inline
torch.__version__

'1.7.1'

In [20]:
#data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

BATCH_SIZE=32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_subset, val_subset = torch.utils.data.random_split(trainset, [40000, 10000])

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_subset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [21]:
from model import Net
net = Net(BATCH_SIZE)

print(net)

def train(epochs):
    min_acc = -np.inf
    epoch_losses = []
    epoch_accs = []
    writer = SummaryWriter()
    for epoch in range(epochs):  # loop over the dataset multiple times

        epoch_loss = 0.0
        epoch_acc = 0.
        val_acc = -np.inf
        val_loss = np.inf
        pbar = tqdm(enumerate(trainloader, 0))
        for i, data in pbar:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(net.device), data[1].to(net.device)

            # zero the parameter gradients
            net.optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = net.loss_fn(outputs, labels)
            loss.backward()
            net.optimizer.step()

            pred = torch.argmax(outputs, axis=1)
            running_acc = (pred == labels).float().sum() / BATCH_SIZE * 100
            running_loss = loss.item()

            epoch_acc += running_acc
            epoch_loss += running_loss

            # print statistics
            pbar.set_description(f'Epoch: {epoch}, Done {(i + 1)/len(trainloader) * 100}%, Loss: {running_loss}, Accuracy: {running_acc}%')
            pbar.update(BATCH_SIZE)
        pbar.close()

        val_acc, val_loss = test(valloader)

        net.scheduler.step(val_loss)
        
        epoch_loss /= len(trainloader)
        epoch_acc /= len(trainloader)
        
        epoch_losses.append(epoch_loss)
        epoch_accs.append(epoch_acc)
        
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('Accuracy/train', epoch_acc, epoch)
        writer.add_scalar('Loss/test', val_loss, epoch)
        writer.add_scalar('Accuracy/test', val_acc, epoch)

        print(f"Avg loss: {colored(str(epoch_loss), 'green')}, Avg accuracy: {colored(str(epoch_acc.item()), 'red')}, Val loss: {val_loss}, Val accuracy: {val_acc}")

        if val_acc > min_acc:
            print(f'Improved val acc from {min_acc} to {val_acc}, saving model')
            min_acc = val_acc
            net.save()
        else:
            print(f'Val acc did not improve from {min_acc}')

    print('Finished Training')
    writer.flush()
    return epoch_accs, epoch_losses

def test(dataloader):
    correct = 0.
    total = 0.
    loss = 0.
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(net.device), data[1].to(net.device)
            outputs = net(images)
            run_loss = net.loss_fn(outputs, labels).item()
            loss += run_loss
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            run_acc = (predicted == labels).sum().item()
            correct += run_acc

    return correct / total * 100, loss / total
    

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (act): Softmax(dim=-1)
)


In [22]:
accs, losses = train(5)
acc, loss = test(testloader)
print(f'Test acc: {acc}%, test loss: {loss}')

Epoch: 0, Done 100.0%, Loss: 1.5590554475784302, Accuracy: 46.875%: : 1250it [00:09, 129.13it/s]               


Avg loss: 1.4953859202861786, Avg accuracy: 46.0, Val loss: 0.041197585606575014, Val accuracy: 52.910000000000004
Improved val acc from -inf to 52.910000000000004, saving model


Epoch: 1, Done 100.0%, Loss: 1.2841237783432007, Accuracy: 59.375%: : 1250it [00:09, 132.73it/s]               


Avg loss: 1.2427513354301452, Avg accuracy: 55.619998931884766, Val loss: 0.03778730186223984, Val accuracy: 57.440000000000005
Improved val acc from 52.910000000000004 to 57.440000000000005, saving model


Epoch: 2, Done 100.0%, Loss: 0.9770117998123169, Accuracy: 68.75%: : 1250it [00:09, 132.96it/s]                


Avg loss: 1.134056944179535, Avg accuracy: 59.71249771118164, Val loss: 0.0365814757168293, Val accuracy: 58.64
Improved val acc from 57.440000000000005 to 58.64, saving model


Epoch: 3, Done 100.0%, Loss: 0.7804016470909119, Accuracy: 62.5%: : 1250it [00:09, 134.68it/s]                 


Avg loss: 1.06313604221344, Avg accuracy: 62.44499969482422, Val loss: 0.03471260567903519, Val accuracy: 61.029999999999994
Improved val acc from 58.64 to 61.029999999999994, saving model


Epoch: 4, Done 100.0%, Loss: 0.8025839328765869, Accuracy: 68.75%: : 1250it [00:09, 134.74it/s]                


Avg loss: 1.0087802947998046, Avg accuracy: 64.12999725341797, Val loss: 0.03410350760221481, Val accuracy: 61.739999999999995
Improved val acc from 61.029999999999994 to 61.739999999999995, saving model
Finished Training
Test acc: 62.129999999999995%, test loss: 0.03365776492357254


In [25]:
%reload_ext tensorboard
%tensorboard --logdir=runs
from tensorboard import notebook
notebook.display(port=80, height=1000)

In [24]:
!kill 21509